In [1]:
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import re
import requests
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

In [2]:
def get_the_top_language(url):
    response = requests.get(url)
    html = response.text
    soup = bs4.BeautifulSoup(html)
    language_container = soup.find(itemprop="keywords")['aria-label']
    language_container = re.sub(r' .*', "", language_container)
    return language_container

In [3]:
get_the_top_language('https://github.com/tesseract-ocr/tesseract')

'C++'

In [4]:
def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data

In [5]:
def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )



In [6]:
def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )

In [7]:
def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


In [8]:
def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }

In [9]:
def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]

In [10]:
def get_the_repos(url):
    response = requests.get(url)
    html = response.text
    soup = bs4.BeautifulSoup(html)
    bleh = soup.find(class_="Link--muted")['href']
    bleh = soup.find_all(class_="Link--muted")
    hold = []
    for item in bleh:
        hold.append(item['href'])
    regexp = r'\/.*\/.*?'
    lister = []
    for item in hold:
        lister.append(re.findall(regexp, item))
    for i in range(0, len(lister)):
        lister[i] = str(lister[i])
        lister[i] = lister[i][3:-3]
    return lister

In [11]:
def get_all_repos(orig_url):
    split_url = orig_url.split('?')
    hold_urls = []
    for i in range(2, 21):
        hold_urls.append(split_url[0] + '?p=' + str(i) + '&' + split_url[1])
    for item in hold_urls:
        all_repos += get_the_repos(item)
    return all_repos

In [12]:
get_all_repos('https://github.com/search?q=repository&type=repositories')

UnboundLocalError: local variable 'all_repos' referenced before assignment

In [ ]:
listuh = ['https://github.com/search?p=20&q=repository&type=repositories', 'https://github.com/search?p=19&q=repository&type=repositories']
for thing in listuh:
    der += get_the_repos(thing)
der

In [ ]:
get_the_repos(listun[12])

In [ ]:
listun